# GENERATIVE AI - REPORT SUMMARIZATION 

In [7]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install torchdata

You should consider upgrading via the 'c:\users\sona\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sona\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sona\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sona\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


Loading datasets, LLM, tokenizer and configurator

In [8]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [9]:
import numpy as np
import pandas as pd

In [17]:
dataset_name = 'EdinburghNLP/xsum'
ds = load_dataset(dataset_name)

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [18]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT REPORT:')
    print(ds['test'][index]['document'])
    print(dash_line)
    print('HUMAN SUMMARY:')
    print(ds['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT REPORT:
But one nation, Russia, was missing.
For the first time in Eurovision history, the host nation barred another country's singer.
That is because in 2015, in violation of Ukrainian border rules, Russia's Julia Samoilova performed in Crimea, the Ukrainian peninsula annexed by Russia a year earlier.
Samoilova suffers from a neural muscular disorder and has used a wheelchair since childhood.
"When the rumours began I might not go, I was so sad," she told the BBC in Moscow. "I thought, how come? This was my dream. When the final decision was taken I didn't believe it. But unfortunately, this is the reality."
"I think it's a stupid reaction," Russian MP Vitaly Milonov tells me. "They're even afraid of such a small girl to enter Kiev."
Even before Ukraine's ban, Mr Milonov

Load the FLAN-T5 model, creating an instance of the AutoModelForSeq2SeqLM class with the .from_pretrained() method.

In [19]:
model_name = 'google/flan-t5-small'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

c:\users\sona\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sona\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

To perform encoding and decoding, you need to work with text in a tokenized form. Tokenization is the process of splitting texts into smaller units that can be processed by the LLM models.

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Testing encoding and decoding

In [22]:
sentence = "How are you Sona?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 571,   33,   25, 3885,    9,   58,    1])

DECODED SENTENCE:
How are you Sona?


# Prompt Engineering

In [25]:
for i, index in enumerate(example_indices):
    document = ds['test'][index]['document']
    summary = ds['test'][index]['summary']

    inputs = tokenizer(document, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{document}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

Token indices sequence length is longer than the specified maximum sequence length for this model (1158 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
But one nation, Russia, was missing.
For the first time in Eurovision history, the host nation barred another country's singer.
That is because in 2015, in violation of Ukrainian border rules, Russia's Julia Samoilova performed in Crimea, the Ukrainian peninsula annexed by Russia a year earlier.
Samoilova suffers from a neural muscular disorder and has used a wheelchair since childhood.
"When the rumours began I might not go, I was so sad," she told the BBC in Moscow. "I thought, how come? This was my dream. When the final decision was taken I didn't believe it. But unfortunately, this is the reality."
"I think it's a stupid reaction," Russian MP Vitaly Milonov tells me. "They're even afraid of such a small girl to enter Kiev."
Even before Ukraine's ban, Mr Milonov

The output is somewhat relevant to the human summary. But could be improved using one-shot or few shot learning

## One-shot Inference

In [28]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        document = ds['test'][index]['document']
        summary = ds['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
document:

{document}

What was going on?
{summary}


"""

    document = ds['test'][example_index_to_summarize]['document']

    prompt += f"""
document:

{document}

What was going on?
"""

    return prompt   

In [29]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


document:

But one nation, Russia, was missing.
For the first time in Eurovision history, the host nation barred another country's singer.
That is because in 2015, in violation of Ukrainian border rules, Russia's Julia Samoilova performed in Crimea, the Ukrainian peninsula annexed by Russia a year earlier.
Samoilova suffers from a neural muscular disorder and has used a wheelchair since childhood.
"When the rumours began I might not go, I was so sad," she told the BBC in Moscow. "I thought, how come? This was my dream. When the final decision was taken I didn't believe it. But unfortunately, this is the reality."
"I think it's a stupid reaction," Russian MP Vitaly Milonov tells me. "They're even afraid of such a small girl to enter Kiev."
Even before Ukraine's ban, Mr Milonov had called for a Russian boycott of Eurovision: "Eurovision became a disgusting socialist nightmare for all these left-wing parties with all their bearded women, or men, with these anti-Christian positions.
"I am

In [30]:
summary = ds['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Harry Potter spin-off Fantastic Beasts and Where To Find Them has had the biggest UK box office opening weekend of the year so far.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
The opening weekend of Harry Potter's first instalment has been the highest opening weekend in the US and Canada, with the opening weekend haul in the US and Canada.


## Few shot

In [31]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


document:

But one nation, Russia, was missing.
For the first time in Eurovision history, the host nation barred another country's singer.
That is because in 2015, in violation of Ukrainian border rules, Russia's Julia Samoilova performed in Crimea, the Ukrainian peninsula annexed by Russia a year earlier.
Samoilova suffers from a neural muscular disorder and has used a wheelchair since childhood.
"When the rumours began I might not go, I was so sad," she told the BBC in Moscow. "I thought, how come? This was my dream. When the final decision was taken I didn't believe it. But unfortunately, this is the reality."
"I think it's a stupid reaction," Russian MP Vitaly Milonov tells me. "They're even afraid of such a small girl to enter Kiev."
Even before Ukraine's ban, Mr Milonov had called for a Russian boycott of Eurovision: "Eurovision became a disgusting socialist nightmare for all these left-wing parties with all their bearded women, or men, with these anti-Christian positions.
"I am

In [37]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
The opening weekend of Harry Potter film has stalled after a massive $3.6mm (£33.9m) box office run.ss, The Stars and Clash are the best film of 2011. to take
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Harry Potter spin-off Fantastic Beasts and Where To Find Them has had the biggest UK box office opening weekend of the year so far.



The few shot inference did not provide much improvement. Anything above 5 or 6 shots is not efficient. Instead we can fine tune the model